In [4]:
import sys
import os
import numpy as np
sys.path.append("..")

from utils.aws import read_json
from constants import *

# Input the filename of the results you want to view
results_filename = "evaluation_results_20240901_143017.json" #input("Enter the filename of the results: ")

# Construct the full S3 key
preproc_run_name = "lex_10"

# Eval params
records = 50


# Load the results from S3
try:
    s3_file_key = f"{S3_DATA_DIR}/{preproc_run_name}/eval/{results_filename}"
    results = read_json(S3_BUCKET_NAME, s3_file_key)
    print("Results loaded successfully.")
    
    # Display the WinDiff score
    print(f"\nWinDiff Score: {results['windiff_score']:.4f}")
    
    # Display the first few predictions and ground truths
    print("\nFirst N predictions and ground truths:")
    for i in range(records):
        print(f"Prediction: {results['predictions'][i]:.4f}, Ground Truth: {results['ground_truths'][i]}")
    
    # Display some statistics
    predictions = np.array(results['predictions'])
    ground_truths = np.array(results['ground_truths'])
    
    print(f"\nTotal number of samples: {len(predictions)}")
    print(f"Total number of positive ground truths: {np.sum(ground_truths)}")
    print(f"Total number of positive predictions: {np.sum(predictions)}")
    print(f"Average prediction: {np.mean(predictions):.4f}")
    print(f"Prediction standard deviation: {np.std(predictions):.4f}")
    print(f"Percentage of positive ground truths: {np.mean(ground_truths) * 100:.2f}%")
    
    # You can add more analysis or visualization here as needed
    
except Exception as e:
    print(f"Error loading results from S3: {str(e)}")

Results loaded successfully.

WinDiff Score: 0.1084

First N predictions and ground truths:
Prediction: 0.0000, Ground Truth: 1
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 1
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 0
Prediction: 0.0000, Ground Truth: 1
Predicti

In [10]:
for i in range(len(results['predictions'])):
    if results['ground_truths'][i] == 1:
        print(f"Prediction: {results['predictions'][i]}, Ground Truth: {results['ground_truths'][i]}")

Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 1, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 1, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Prediction: 0, Ground Truth: 1
Predicti